In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
import warnings
import json
import re
import matplotlib.patches as mpatches

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [ ]:
#Add functions for Univariate analysis
def describe_numerical_col(df, col_name):
    info = df[[col_name]].describe().to_dict()[col_name]
    info['shapiro'] = f'{stats.shapiro(df[col_name])[1]: .5f}'
    info['normal'] = float(info['shapiro']) > 0.05
    info['missing'] = df[col_name].isna().sum()
    info['skew'] = f'{stats.skew(df[col_name]):.5f}'
    info['type'] = ('slight ' if info['normal'] else '') + \
                   ('right(positive)' if float(info['skew']) > 0 else 'left(negative)') + '-skew'

    fig, ax = plt.subplots(2, 2, figsize=(16, 7), gridspec_kw={'height_ratios': (.85, .15)})
    sns.histplot(df[col_name], kde=True, ax=ax[0, 0], color='#55A868')
    sns.boxplot(df[col_name], orient='h', ax=ax[1, 0], color="#5583A8")
    # محاسبه outliers
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    lower_outliers = df[col_name][df[col_name] < lower_bound]
    upper_outliers = df[col_name][df[col_name] > upper_bound]
    total_outliers = len(lower_outliers) + len(upper_outliers)
    percent_outliers = 100 * total_outliers / len(df[col_name])
    label_text = (
        f"Lower Outliers: {len(lower_outliers)}\n"
        f"Upper Outliers: {len(upper_outliers)}\n"
        f"Total: {total_outliers} ({percent_outliers:.1f}%)"
    )
    patch = mpatches.Patch(color='skyblue', label=label_text)
    ax[1, 0].legend(handles=[patch], fontsize=12, loc='upper left', bbox_to_anchor=(1.05, 1))
    # پایان outliers
    counts, bin_edges = np.histogram(df[col_name], bins=10, density=True)
    pdf = counts / (sum(counts))
    cdf = np.cumsum(pdf)
    ax[1, 1] = plt.subplot(122)
    plt.plot(bin_edges[1:], pdf, label='PDF')
    plt.plot(bin_edges[1:], cdf, label='CDF')
    plt.legend()
    ax[0, 0].set_xticklabels([])
    ax[1, 0].set_yticklabels([])
    ax[0, 0].set_xlabel('')
    ax[0, 0].set_ylabel('Count')
    fig.suptitle(col_name, fontsize=30)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    info_df = pd.DataFrame.from_dict(info, orient='index', columns=[''])
    print('=' * 18 + ' ' + col_name + ' ' + '=' * 18)
    print(info_df)
    print('=' * 40)


def categorize_numerical_col(col_name, bins, bins_name):
    new_col = f'{col_name}_categorized'
    df[new_col] = pd.cut(df[col_name], bins=bins, labels=bins_name)


def describe_categorical_col(df, col_name):
    counts = pd.DataFrame(df[col_name].value_counts()).reset_index()
    counts.columns = ['Group', 'Count']
    total = sum(counts['Count'])
    counts['%'] = (counts['Count'] / total * 100).round(2)

    fig = px.pie(
        counts,
        names='Group',
        values='Count',
        title=f'<b>Distribution of {col_name} in each group</b>',
        color='Group',
        hole=0.3
    )

    fig.update_traces(
        textposition='inside',
        textinfo='percent+label',

    )

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Groups',
        font=dict(family="Arial, sans-serif", size=14)
    )
    fig.show()
    print('=' * 18 + ' ' + col_name + ' ' + '=' * 18)
    print(counts)
    print('=' * 40)
    # plt.figure(figsize=(12, 7))
    # ax = sns.barplot(
    #     data=counts,
    #     x=counts[col_name],
    #     y=counts['count'],
    #     palette='viridis'
    # )

    # for i in range(len(ax.patches)):
    #     p = ax.patches[i]
    #     percentage = counts['percents'][i]
    #     ax.annotate(f'{percentage:.2f}%',
    #                 (p.get_x() + p.get_width()/2., p.get_height()),
    #                 ha='center', va='center',
    #                 fontsize=11, color='black',
    #                 xytext=(0, 10),
    #                 textcoords='offset points')

    # plt.ylim(0, max(counts['count']) * 1.15)
    # ax.set_xlabel('')
    # ax.set_title(f'Distribution of {col_name} in each group')
    # plt.show()


In [ ]:
def describe_num_num_relationship(df, col1, col2):
    """تحلیل رابطه بین دو متغیر عددی"""
    # آزمون نرمال بودن با شاپیرو
    shapiro1_stat, shapiro1_p = stats.shapiro(df[col1])
    shapiro2_stat, shapiro2_p = stats.shapiro(df[col2])

    # تشخیص نرمال بودن (p > 0.05 یعنی نرمال)
    is_normal1 = shapiro1_p > 0.05
    is_normal2 = shapiro2_p > 0.05
    both_normal = is_normal1 and is_normal2

    # انتخاب آزمون مناسب
    if both_normal:
        # استفاده از پیرسون
        corr_stat, p_value = stats.pearsonr(df[col1], df[col2])
        test_used = 'Pearson'
        correlation = corr_stat
    else:
        # استفاده از اسپیرمن
        corr_stat, p_value = stats.spearmanr(df[col1], df[col2])
        test_used = 'Spearman'
        correlation = corr_stat

    r2_score = correlation ** 2

    info = {
        'shapiro_p_col1': f'{shapiro1_p:.5f}',
        'shapiro_p_col2': f'{shapiro2_p:.5f}',
        'col1_normal': is_normal1,
        'col2_normal': is_normal2,
        'test_used': test_used,
        'correlation': f'{correlation:.5f}',
        'r_squared': f'{r2_score:.5f}',
        'p_value': f'{p_value:.5f}',
        'significant': p_value < 0.05,
        'relationship_strength': 'strong' if abs(correlation) > 0.7 else 'moderate' if abs(
            correlation) > 0.3 else 'weak',
        'relationship_direction': 'positive' if correlation > 0 else 'negative'
    }

    # رسم نمودار
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(f'{col1} vs {col2} - {test_used} Correlation', fontsize=14)

    # Scatter Plot
    axes[0].scatter(df[col1], df[col2],
                    s=60, alpha=0.7, color='blue', edgecolors='black', linewidth=0.5)
    axes[0].set_title('Scatter Plot')
    axes[0].set_xlabel(col1)
    axes[0].set_ylabel(col2)
    axes[0].grid(True, alpha=0.3)

    # Hexbin Plot
    axes[1].hexbin(df[col1], df[col2], gridsize=20, cmap='Blues', mincnt=1)
    axes[1].set_title('Hexbin Plot')
    axes[1].set_xlabel(col1)
    axes[1].set_ylabel(col2)
    axes[1].yaxis.set_label_position("right")

    plt.tight_layout()
    plt.show()

    info_df = pd.DataFrame.from_dict(info, orient='index', columns=[''])
    print('=' * 15 + f' {col1} vs {col2} ' + '=' * 15)
    print(info_df)
    print('=' * 50)


def describe_cat_num_relationship(df, cat_col, num_col):
    """تحلیل رابطه بین متغیر کیفی و کمی"""
    # محاسبه آمار
    groups = df.groupby(cat_col)[num_col].describe()

    # آزمون نرمال بودن با شاپیرو برای هر گروه
    normality_results = {}
    group_data = []
    all_normal = True

    for name, group in df.groupby(cat_col):
        if len(group) >= 3:  # شاپیرو حداقل 3 داده نیاز داره
            shapiro_stat, shapiro_p = stats.shapiro(group[num_col])
            is_normal = shapiro_p > 0.05
            normality_results[name] = {
                'shapiro_p': f'{shapiro_p:.5f}',
                'is_normal': is_normal
            }
            if not is_normal:
                all_normal = False
        else:
            normality_results[name] = {
                'shapiro_p': 'N/A (too few data)',
                'is_normal': False
            }
            all_normal = False

        group_data.append(group[num_col].values)

    # انتخاب آزمون مناسب
    if all_normal:
        # استفاده از ANOVA معمولی
        f_stat, p_value = stats.f_oneway(*group_data)
        test_used = 'ANOVA'
    else:
        # استفاده از Welch's ANOVA
        try:
            from scipy.stats import alexandergovern
            # اگر alexandergovern موجود نباشه، از kruskal استفاده می‌کنیم
            result = alexandergovern(*group_data)
            f_stat, p_value = result.statistic, result.pvalue
            test_used = "Welch's ANOVA"
        except ImportError:
            # fallback به Kruskal-Wallis (non-parametric)
            f_stat, p_value = stats.kruskal(*group_data)
            test_used = 'Kruskal-Wallis (non-parametric)'

    info = {
        'test_used': test_used,
        'all_groups_normal': all_normal,
        'f_statistic': f'{f_stat:.5f}',
        'p_value': f'{p_value:.5f}',
        'significant_difference': p_value < 0.05,
        'num_categories': df[cat_col].nunique(),
        'total_observations': len(df)
    }

    # رسم نمودار
    fig = plt.subplots(2, 2, figsize=(14, 10))

    # Box plot
    ax1 = plt.subplot(221)
    sns.boxplot(data=df, x=cat_col, y=num_col)
    ax1.set_title(f'{num_col} by {cat_col}')
    plt.xticks(rotation=45)

    # Violin plot
    ax2 = plt.subplot(222)
    sns.violinplot(data=df, x=cat_col, y=num_col)
    ax2.set_title(f'{num_col} Distribution by {cat_col}')
    plt.xticks(rotation=45)

    # Bar plot of means
    ax3 = plt.subplot(223)
    means = df.groupby(cat_col)[num_col].mean().sort_values(ascending=False)
    sns.barplot(x=means.index, y=means.values)
    ax3.set_title(f'Mean {num_col} by {cat_col}')
    plt.xticks(rotation=45)

    # Count plot of categories
    ax4 = plt.subplot(224)
    sns.countplot(data=df, x=cat_col)
    ax4.set_title(f'Count of {cat_col}')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

    print('=' * 15 + f' {cat_col} vs {num_col} ' + '=' * 15)
    print("Group Statistics:")
    print(groups)
    print(f"\nNormality Test Results (Shapiro-Wilk):")
    for group_name, result in normality_results.items():
        print(f"{group_name}: p-value = {result['shapiro_p']}, Normal = {result['is_normal']}")
    print(f"\n{test_used} Test Results:")
    info_df = pd.DataFrame.from_dict(info, orient='index', columns=[''])
    print(info_df)
    print('=' * 50)


def describe_cat_cat_relationship(df, col1, col2):
    """تحلیل رابطه بین دو متغیر کیفی"""
    # جدول تطبیقی
    contingency_table = pd.crosstab(df[col1], df[col2])

    # آزمون کای دو
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

    # ضریب کرامر V
    n = contingency_table.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

    info = {
        'chi2_statistic': f'{chi2:.5f}',
        'p_value': f'{p_value:.5f}',
        'degrees_of_freedom': dof,
        'cramers_v': f'{cramers_v:.5f}',
        'association_strength': 'strong' if cramers_v > 0.3 else 'moderate' if cramers_v > 0.1 else 'weak',
        'significant_association': p_value < 0.05
    }

    # رسم نمودار
    fig = plt.subplots(2, 2, figsize=(14, 10))

    # Heatmap of contingency table
    ax1 = plt.subplot(221)
    sns.heatmap(contingency_table, annot=True, fmt='d', cmap='Blues')
    ax1.set_title(f'Contingency Table: {col1} vs {col2}')

    # Stacked bar chart
    ax2 = plt.subplot(222)
    contingency_table.plot(kind='bar', stacked=True, ax=ax2)
    ax2.set_title(f'Stacked Bar: {col1} vs {col2}')
    plt.xticks(rotation=45)

    # Normalized heatmap (percentages)
    ax3 = plt.subplot(223)
    normalized = contingency_table.div(contingency_table.sum(axis=1), axis=0)
    sns.heatmap(normalized, annot=True, fmt='.2%', cmap='Oranges')
    ax3.set_title(f'Normalized Contingency Table')

    # Count plots side by side
    ax4 = plt.subplot(224)
    df_melted = df[[col1, col2]].melt()
    sns.countplot(data=df_melted, x='value', hue='variable')
    ax4.set_title(f'Count Comparison')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

    print('=' * 15 + f' {col1} vs {col2} ' + '=' * 15)
    print("Contingency Table:")
    print(contingency_table)
    print("\nChi-square Test Results:")
    info_df = pd.DataFrame.from_dict(info, orient='index', columns=[''])
    print(info_df)
    print('=' * 50)


def describe_target_relationship(df, feature_col, target_col):
    """تحلیل رابطه هر متغیر با متغیر هدف"""
    if df[feature_col].dtype in ['object', 'category']:
        if df[target_col].dtype in ['object', 'category']:
            describe_cat_cat_relationship(df, feature_col, target_col)
        else:
            describe_cat_num_relationship(df, feature_col, target_col)
    else:
        if df[target_col].dtype in ['object', 'category']:
            describe_cat_num_relationship(df, target_col, feature_col)
        else:
            describe_num_num_relationship(df, feature_col, target_col)



In [ ]:
df = pd.read_csv('marketing.csv')

In [ ]:
df.head(20)

In [ ]:
df.info()

In [ ]:
df.describe()

<h1>Univariate Analysis & Data Cleanning</h1>

<h3>AdSpend</h3>

In [ ]:
#تبدیل ستون به تایپ عددی و اصلاح داده ها
df['AdSpend'] = df['AdSpend'].astype(str).str.replace(r'[^\d\.]', '', regex=True).astype(float)
df.describe()


In [ ]:
describe_numerical_col(df, 'AdSpend')

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل ستون AdSpend</h2>
  <p>ستون <strong>AdSpend</strong> نشان‌دهنده میزان هزینه تبلیغات برای هر رکورد است و یکی از متغیرهای کلیدی در تحلیل بازاریابی محسوب می‌شود. داده‌ها کامل و بدون مقدار گمشده‌اند و در بازه‌ای نسبتاً گسترده از حدود ۱۰۰ تا ۱۰,۰۰۰ واحد قرار دارند.</p>
  <p>میانگین و میانه بسیار نزدیک‌اند که به ظاهر متقارن بودن توزیع اشاره دارد. با این حال، آزمون Shapiro مقدار p را صفر گزارش کرده و نرمال بودن را رد می‌کند. چولگی مثبت بسیار جزئی (۰.۰۰۳۱۵) وجود دارد، اما در عمل توزیع تقریباً متقارن است. منحنی‌های PDF و CDF نیز شکل منظم و پیوسته‌ای دارند و هیچ نقطه پرت شناسایی نشده است.</p>
  <p>در مجموع، توزیع AdSpend پایدار، متعادل و غیرنرمال است. برای تحلیل‌های آماری، استفاده از روش‌های مقاوم یا ناپارامتری توصیه می‌شود. این ستون می‌تواند نقش مهمی در بررسی رابطه بین هزینه تبلیغات و نرخ تبدیل ایفا کند.</p>
</div>

In [ ]:
bins_AdSpend = [0, 2500, 5000, 7500, 10000]
bin_labels_AdSpend = ['Very Low Budget', 'Low Budget', 'Medium Budget', 'High Budget']
categorize_numerical_col('AdSpend', bins_AdSpend, bin_labels_AdSpend)
describe_categorical_col(df, 'AdSpend_categorized')

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">توضیح دسته‌های هزینه تبلیغات (AdSpend)</h2>
  <p><strong>Very Low Budget:</strong> زیر ۲۵۰۰ واحد، نشان‌دهنده کمپین‌های کم‌هزینه یا آزمایشی</p>
  <p><strong>Low Budget:</strong> بین ۲۵۰۰ تا ۵۰۰۰ واحد، سطح سرمایه‌گذاری پایه ولی قابل توجه</p>
  <p><strong>Medium Budget:</strong> بین ۵۰۰۰ تا ۷۵۰۰ واحد، بودجه متعادل برای کمپین‌های استاندارد</p>
  <p><strong>High Budget:</strong> بالای ۷۵۰۰ واحد، نشان‌دهنده سرمایه‌گذاری سنگین و هدف‌گذاری گسترده</p>
</div>

<h3>ClickThroughRate</h3>


In [ ]:
describe_numerical_col(df, "ClickThroughRate")

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل ستون ClickThroughRate</h2>
  <p>ستون <strong>ClickThroughRate</strong> (نرخ کلیک) نشان‌دهنده میزان تعامل کاربران با تبلیغات است و یکی از شاخص‌های کلیدی در ارزیابی اثربخشی کمپین‌ها محسوب می‌شود. داده‌ها کامل و بدون مقدار گمشده‌اند و در بازه‌ای بین حدود ۰.۰۱ تا ۰.۳۰ قرار دارند.</p>
  <p>میانگین و میانه تقریباً برابرند که به ظاهر متقارن بودن توزیع اشاره دارد. با این حال، آزمون Shapiro مقدار p را صفر گزارش کرده و نرمال بودن را رد می‌کند. چولگی منفی بسیار جزئی (−۰.۰۱۱۵۸) وجود دارد، اما در عمل توزیع تقریباً متقارن است.</p>
  <p>منحنی‌های PDF و CDF نیز شکل منظم و پیوسته‌ای دارند، بدون جهش یا دم‌های کشیده. همچنین هیچ نقطه پرت شناسایی نشده که نشان‌دهنده پایداری و یکنواختی نسبی داده‌هاست. با توجه به غیرنرمال بودن توزیع، استفاده از آزمون‌های ناپارامتری برای تحلیل‌های آماری توصیه می‌شود.</p>
</div>

<h3>ConversionRate</h3>

In [ ]:
describe_numerical_col(df, "ConversionRate")

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل ستون ConversionRate</h2>
  <p>ستون <strong>ConversionRate</strong> نشان‌دهنده نرخ تبدیل کاربران است—یعنی درصدی از افراد که پس از تعامل با تبلیغ، اقدام مورد نظر را انجام داده‌اند. داده‌ها کامل و بدون مقدار گمشده‌اند و در بازه‌ای بین حدود ۰.۰۱ تا ۰.۲۰ قرار دارند.</p>
  <p>میانگین و میانه بسیار نزدیک‌اند، اما آزمون Shapiro مقدار p را صفر گزارش کرده و نرمال بودن توزیع را رد می‌کند. چولگی مثبت جزئی (۰.۰۷۲۷۸) وجود دارد که نشان‌دهنده کشیدگی خفیف به سمت مقادیر بالاتر است. منحنی PDF نسبتاً یکنواخت و CDF پیوسته و منظم است. نبود نقاط پرت نشان می‌دهد که داده‌ها پایدار و بدون نوسان شدید هستند.</p>
  <p>با توجه به غیرنرمال بودن توزیع، استفاده از آزمون‌های آماری ناپارامتری برای تحلیل‌های بعدی توصیه می‌شود. این ستون برای ارزیابی عملکرد کمپین‌ها و شناسایی عوامل مؤثر بر تبدیل، بسیار کلیدی است.</p>
</div>

In [ ]:
bins_ConversionRate = [0, 0.05, 0.10, 0.15, 0.20]
bin_labels_ConversionRate = ['Very Low Conversion', 'Low Conversion', 'Moderate Conversion', 'High Conversion']
categorize_numerical_col('ConversionRate', bins_ConversionRate, bin_labels_ConversionRate)
describe_categorical_col(df, "ConversionRate_categorized")

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">توضیح دسته‌های نرخ تبدیل (ConversionRate)</h2>
  <p><strong>Very Low Conversion:</strong> زیر ۵٪، معمولاً نشان‌دهنده ضعف در جذب یا هدف‌گذاری</p>
  <p><strong>Low Conversion:</strong> بین ۵٪ تا ۱۰٪، عملکرد قابل قبول ولی نه چشمگیر</p>
  <p><strong>Moderate Conversion:</strong> بین ۱۰٪ تا ۱۵٪، نرخ تبدیل مناسب و مؤثر</p>
  <p><strong>High Conversion:</strong> بالای ۱۵٪، نشان‌دهنده کمپین‌های موفق و تعامل بالا</p>
</div>

<h3>PreviousPurchases</h3>


In [ ]:
describe_numerical_col(df, "PreviousPurchases")

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل ستون PreviousPurchases</h2>
  <p>ستون <strong>PreviousPurchases</strong> نشان‌دهنده تعداد خریدهای قبلی هر مشتری است و شاخصی مهم برای سنجش وفاداری یا سابقه تعامل با برند محسوب می‌شود. داده‌ها کامل و بدون مقدار گمشده‌اند و در بازه‌ای بین ۰ تا ۹ خرید قرار دارند.</p>
  <p>میانگین و میانه نزدیک‌اند (میانگین ≈ ۴.۵۴، میانه = ۵) که به ظاهر متقارن بودن توزیع اشاره دارد. با این حال، آزمون Shapiro مقدار p را صفر گزارش کرده و نرمال بودن توزیع را رد می‌کند. چولگی منفی بسیار جزئی (−۰.۰۱۲۸۸) وجود دارد، اما در عمل توزیع پایدار و متعادل است.</p>
  <p>هیچ نقطه پرت شناسایی نشده و منحنی‌های PDF و CDF شکل منظم و پیوسته‌ای دارند. با توجه به غیرنرمال بودن، استفاده از آزمون‌های آماری ناپارامتری توصیه می‌شود. این ستون می‌تواند در تحلیل رفتار خرید و پیش‌بینی نرخ تبدیل نقش کلیدی داشته باشد.</p>
</div>

In [ ]:
bins_PreviousPurchases = [0, 3, 7, 10]
bin_labels_PreviousPurchases = ['0–2 Purchases', '3–6 Purchases', '7–9 Purchases']
categorize_numerical_col('PreviousPurchases', bins_PreviousPurchases, bin_labels_PreviousPurchases)
describe_categorical_col(df, 'PreviousPurchases_categorized')

<h1>Bivariate</h1>

<h3>AdSpend</h3>

In [ ]:
describe_cat_num_relationship(df, "Conversion", "AdSpend")

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل نمودارها</h2>
  <p>نمودارهای ارائه‌شده چهار دیدگاه مکمل از رابطه بین هزینه تبلیغات (AdSpend) و تبدیل (Conversion) ارائه می‌دهند:</p>
  <p><strong>Box Plot:</strong> میانه هزینه تبلیغات برای گروه تبدیل‌شده (Conversion = 1) به‌وضوح بالاتر از گروه غیرتبدیل‌شده (Conversion = 0) است. دامنه بین چارک‌ها در گروه تبدیل‌شده گسترده‌تر است که نشان‌دهنده تنوع بیشتر در بودجه‌های مؤثر است.</p>
  <p><strong>Violin Plot:</strong> تراکم داده‌ها در گروه تبدیل‌شده بیشتر در محدوده‌های بالاتر هزینه متمرکز شده که بیانگر احتمال بیشتر تبدیل در کمپین‌های پرهزینه‌تر است.</p>
  <p><strong>Bar Plot میانگین AdSpend:</strong> میانگین هزینه تبلیغات برای گروه تبدیل‌شده بیشتر از گروه غیرتبدیل‌شده است و این تفاوت در آمار عددی نیز تأیید شده است.</p>
  <p><strong>Bar Plot تعداد Conversion:</strong> تعداد نمونه‌های تبدیل‌شده بیشتر از غیرتبدیل‌شده است که می‌تواند ناشی از هدف‌گذاری دقیق‌تر یا کیفیت بالاتر کمپین‌ها باشد.</p>

  <h2 style="color:#2c3e50;">تحلیل آماری مقادیر</h2>
  <p>میانگین هزینه تبلیغات برای گروه تبدیل‌شده حدود ۵۱۴۸ واحد و برای گروه غیرتبدیل‌شده حدود ۴۱۷۷ واحد است. میانه نیز همین روند را تأیید می‌کند: ۵۲۴۹ در برابر ۳۷۱۳. آزمون Welch’s ANOVA با مقدار f برابر با ۲۷.۳۹ و p-value صفر، اختلاف معناداری بین میانگین‌های دو گروه را تأیید می‌کند.</p>

  <h2 style="color:#2c3e50;">جمع‌بندی</h2>
  <p>تحلیل تصویری و آماری هر دو نشان می‌دهند که هزینه تبلیغات بالاتر به‌طور معناداری با نرخ تبدیل بیشتر همراه است. این رابطه هم از نظر آماری معتبر است و هم در نمودارها به‌وضوح قابل مشاهده است. بنابراین، سرمایه‌گذاری بیشتر در تبلیغات—حداقل در این مجموعه داده—با اثربخشی بالاتر کمپین‌ها و نرخ تبدیل بیشتر همراه است.</p>
</div>

In [ ]:
describe_cat_cat_relationship(df, "AdSpend_categorized", "Conversion")

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل نمودارها</h2>
  <p>نمودارهای ارائه‌شده رابطه بین دسته‌های هزینه تبلیغات (<strong>AdSpend_categorized</strong>) و تبدیل کاربران (<strong>Conversion</strong>) را از زوایای مختلف بررسی می‌کنند:</p>
  <p><strong>جدول فراوانی:</strong> نشان می‌دهد که در دسته‌های با بودجه بالاتر (Medium و High)، تعداد تبدیل‌ها به‌طور قابل توجهی بیشتر از دسته‌های با بودجه پایین‌تر است. به‌ویژه دسته Medium Budget با ۴۹۵ تبدیل، بیشترین سهم را دارد.</p>
  <p><strong>نمودار میله‌ای انباشته:</strong> توزیع تبدیل و عدم تبدیل در هر دسته را نمایش می‌دهد. سهم تبدیل در دسته‌های با بودجه بیشتر، به‌وضوح بالاتر است.</p>
  <p><strong>جدول نرمال‌شده:</strong> درصد تبدیل در دسته High Budget برابر با ۵۴.۸۸٪ است، در حالی که این مقدار در دسته Very Low Budget فقط ۴۴.۳۵٪ است. این اختلاف نشان‌دهنده رابطه مثبت بین سطح بودجه تبلیغاتی و احتمال تبدیل است.</p>
  <p><strong>نمودار مقایسه تعداد:</strong> تعداد تبدیل‌ها در دسته‌های با بودجه بیشتر به‌طور قابل توجهی بالاتر است، که می‌تواند نشانه‌ای از اثربخشی سرمایه‌گذاری تبلیغاتی باشد.</p>

  <h2 style="color:#2c3e50;">تحلیل آماری مقادیر</h2>
  <p>آزمون <strong>Chi-square</strong> با مقدار آماره ۳۷.۰۱ و p-value برابر با ۰.۰۰۰۰۰، وجود ارتباط معنادار بین دسته‌های AdSpend و Conversion را تأیید می‌کند. مقدار <strong>Cramér's V</strong> برابر با ۰.۱۳۵۷۰ است که نشان‌دهنده قدرت ارتباط متوسط است.</p>

  <h2 style="color:#2c3e50;">جمع‌بندی</h2>
  <p>هم نمودارها و هم آزمون آماری نشان می‌دهند که افزایش بودجه تبلیغاتی با نرخ تبدیل بالاتر همراه است. این رابطه از نظر آماری معتبر بوده و در نمودارها نیز به‌وضوح قابل مشاهده است. بنابراین، سطح سرمایه‌گذاری در تبلیغات می‌تواند نقش مؤثری در موفقیت کمپین‌ها ایفا کند.</p>
</div>

In [ ]:
describe_cat_num_relationship(df, "Conversion", "ClickThroughRate")


<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل نمودارها</h2>
  <p>چهار نمودار ارائه‌شده دیدگاه‌های مکملی از ارتباط بین نرخ کلیک (<strong>ClickThroughRate</strong>) و تبدیل کاربران (<strong>Conversion</strong>) ارائه می‌دهند:</p>
  <p><strong>Box Plot:</strong> میانه نرخ کلیک در گروه تبدیل‌شده (Conversion = 1) به‌وضوح بالاتر از گروه غیرتبدیل‌شده است. دامنه بین چارک‌ها در گروه تبدیل‌شده گسترده‌تر است که بیانگر تنوع بیشتر در تعامل کاربران است.</p>
  <p><strong>Violin Plot:</strong> تراکم داده‌ها در گروه تبدیل‌شده بیشتر در محدوده‌های بالاتر نرخ کلیک دیده می‌شود که نشان‌دهنده ارتباط مثبت بین تعامل و تبدیل است.</p>
  <p><strong>Bar Plot میانگین ClickThroughRate:</strong> میانگین نرخ کلیک برای گروه تبدیل‌شده (۰.۱۶۰۸) بیشتر از گروه غیرتبدیل‌شده (۰.۱۲۵۱) است.</p>
  <p><strong>Bar Plot تعداد Conversion:</strong> تعداد نمونه‌های تبدیل‌شده بسیار بیشتر از نمونه‌های غیرتبدیل‌شده است که می‌تواند ناشی از اثربخشی کمپین‌ها یا هدف‌گذاری دقیق‌تر باشد.</p>

  <h2 style="color:#2c3e50;">تحلیل آماری مقادیر</h2>
  <p>میانگین نرخ کلیک برای گروه تبدیل‌شده حدود ۰.۱۶۰۸ و برای گروه غیرتبدیل‌شده حدود ۰.۱۲۵۱ است. میانه نیز همین روند را تأیید می‌کند: ۰.۱۶۲۴ در برابر ۰.۰۹۴۷. آزمون نرمال بودن (Shapiro-Wilk) برای هر دو گروه مقدار p را صفر گزارش کرده که نشان‌دهنده غیرنرمال بودن توزیع‌هاست. آزمون Welch’s ANOVA با مقدار f برابر با ۳۵.۲۸ و p-value صفر، اختلاف معناداری بین میانگین‌های دو گروه را تأیید می‌کند.</p>

  <h2 style="color:#2c3e50;">جمع‌بندی</h2>
  <p>هم تحلیل تصویری و هم تحلیل آماری نشان می‌دهند که نرخ کلیک بالاتر به‌طور معناداری با نرخ تبدیل بیشتر همراه است. این رابطه از نظر آماری معتبر بوده و در نمودارها نیز به‌وضوح قابل مشاهده است. بنابراین، می‌توان نتیجه گرفت که افزایش تعامل کاربران با تبلیغات—حداقل در این مجموعه داده—با احتمال تبدیل بیشتر همراه است.</p>
</div>

In [ ]:
describe_cat_cat_relationship(df, "PreviousPurchases_categorized","Conversion")

<div style="font-family:Tahoma, sans-serif; line-height:1.8; direction:rtl; text-align:right; padding:20px;">
  <h2 style="color:#2c3e50;">تحلیل نمودارها</h2>
  <p>نمودارهای ارائه‌شده رابطه بین تعداد خریدهای قبلی مشتریان (<strong>PreviousPurchases_categorized</strong>) و وضعیت تبدیل (<strong>Conversion</strong>) را از زوایای مختلف بررسی می‌کنند:</p>
  <p><strong>جدول فراوانی:</strong> نشان می‌دهد که در دسته ۷–۹ خرید، تعداد تبدیل‌ها (۳۸۱ مورد) نسبت به دسته‌های دیگر بیشتر است. این روند در دسته‌های ۳–۶ و ۰–۲ نیز قابل مشاهده است ولی با شدت کمتر.</p>
  <p><strong>نمودار میله‌ای انباشته:</strong> توزیع تبدیل و عدم تبدیل در هر دسته را نمایش می‌دهد. سهم تبدیل در دسته‌های با خرید بیشتر، به‌وضوح بالاتر است.</p>
  <p><strong>جدول نرمال‌شده:</strong> درصد تبدیل در دسته ۷–۹ خرید برابر با ۱۸.۳۹٪ است، در حالی که این مقدار در دسته ۰–۲ خرید فقط ۱۳.۰۸٪ است. این اختلاف نشان‌دهنده رابطه مثبت بین سابقه خرید و احتمال تبدیل است.</p>
  <p><strong>نمودار مقایسه تعداد:</strong> تعداد تبدیل‌ها در دسته‌های با خرید بیشتر به‌طور قابل توجهی بالاتر است، که می‌تواند نشانه‌ای از وفاداری و آمادگی بیشتر برای اقدام باشد.</p>

  <h2 style="color:#2c3e50;">تحلیل آماری مقادیر</h2>
  <p>آزمون <strong>Chi-square</strong> با مقدار آماره ۸.۶۸ و p-value برابر با ۰.۰۱۲۹۸، وجود ارتباط معنادار بین دسته‌های PreviousPurchases و Conversion را تأیید می‌کند. مقدار <strong>Cramér's V</strong> برابر با ۰.۰۶۹۴۹ است که نشان‌دهنده قدرت ارتباط ضعیف ولی قابل توجه است.</p>

  <h2 style="color:#2c3e50;">جمع‌بندی</h2>
  <p>هم نمودارها و هم آزمون آماری نشان می‌دهند که مشتریانی با سابقه خرید بیشتر، احتمال تبدیل بالاتری دارند. اگرچه قدرت ارتباط ضعیف است، اما از نظر آماری معنادار بوده و می‌تواند در هدف‌گذاری کمپین‌ها و تحلیل رفتار مشتریان نقش مؤثری ایفا کند.</p>
</div>